In [1]:
import urllib.request
import ssl
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [162]:
import pandas as pd

df = pd.read_excel('./data/예금거래내역조회_20210106.xls', header=4)
df.head()

,거래일시,거래구분,기재내용,입금금액,출금금액,잔액,상대 예금주명,메모
0,2019.04.25 14:13,체크결제,파리바게트 화양리점,NaN,6150,20210,NaN,NaN
1,2019.04.25 20:33,체크결제,GS25군자행운점,NaN,10000,10210,NaN,NaN
2,2019.04.26 02:22,체크결제,비지에프리테일세종대,NaN,4000,6210,NaN,NaN
3,2019.04.26 04:23,체크결제,GS25화양중앙점,NaN,4320,1890,NaN,NaN
4,2019.04.29 10:38,체크결제,씨브이에스넷(주),NaN,5500,40190,NaN,NaN


In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1145 entries, 0 to 1144
Data columns (total 8 columns):
거래일시       1145 non-null object
거래구분       1145 non-null object
기재내용       1145 non-null object
입금금액       0 non-null float64
출금금액       1145 non-null int64
잔액         1145 non-null int64
상대 예금주명    211 non-null object
메모         0 non-null float64
dtypes: float64(2), int64(2), object(4)
memory usage: 71.7+ KB


In [164]:
df.rename(columns={"메모":"Category"}, inplace=True)
df.head()

,거래일시,거래구분,기재내용,입금금액,출금금액,잔액,상대 예금주명,Category
0,2019.04.25 14:13,체크결제,파리바게트 화양리점,NaN,6150,20210,NaN,NaN
1,2019.04.25 20:33,체크결제,GS25군자행운점,NaN,10000,10210,NaN,NaN
2,2019.04.26 02:22,체크결제,비지에프리테일세종대,NaN,4000,6210,NaN,NaN
3,2019.04.26 04:23,체크결제,GS25화양중앙점,NaN,4320,1890,NaN,NaN
4,2019.04.29 10:38,체크결제,씨브이에스넷(주),NaN,5500,40190,NaN,NaN


In [165]:
# train 데이터셋 컬럼: 날짜, 상호명, 카테고리, 금액
df_copy = df.copy()
train_df = df[['거래일시','기재내용','출금금액', 'Category']]
train_df

,거래일시,기재내용,출금금액,Category
0,2019.04.25 14:13,파리바게트 화양리점,6150,NaN
1,2019.04.25 20:33,GS25군자행운점,10000,NaN
2,2019.04.26 02:22,비지에프리테일세종대,4000,NaN
3,2019.04.26 04:23,GS25화양중앙점,4320,NaN
4,2019.04.29 10:38,씨브이에스넷(주),5500,NaN
...,...,...,...,...
1140,2020.04.24 14:35,진현영,11000,NaN
1141,2020.04.24 19:40,네이버페이,9000,NaN
1142,2020.04.25 02:34,Apple,3300,NaN
1143,2020.04.25 12:07,네이버페이,16600,NaN


### 계좌이체에 해당되는 내역 제거

In [166]:
drop1_index = train_df[train_df['기재내용'].map(len) == 3].index
train_df['기재내용'][drop1_index].unique()

array(['김은희', '진현영', '이재성', '이준범', '김향미', '김영현', '새로이', '최세은', '홈마트',
       '이주희', '주아영', '가츠시', '황지혜', '차지이', '이장후', '유정화', '그리네', '배고팡',
       '윤지의', '이희진', '김한비', '박도형', '배창현', '김윤환', '팔공티', '이혜린', '정아현',
       '박병오', '띵카롱', '차송하', '박효정', '김은숙', '사계절', '아비꼬', '임소현', '백남문',
       '남한나', '송준희', '김유진', '한초원', '정명주', '한예진', '윤정은', '전대숙', '김승현',
       '세종원', '한은비', '손지연', '조정인', '김지오', '하오츠', '노유정', '카카오', '김진희',
       '심비어', '김희선', '장현진', '신민철', '변준영', '양주영', '양분식', '도네누', '카페롱',
       '천미향', '신동희', '김두열', '하얗다', '더호떡', '정홍비', '권규경', '이승연', '꼽사리',
       '박민서', '홍짱H', '연약국', '송승훈', '강양기', '시너리', '개미집', '차승현', '류영구',
       '이시현', '위메프', '김민지', '박태정', '민수현'], dtype=object)

In [167]:
train_df['기재내용'][drop1_index].unique()

array(['김은희', '진현영', '이재성', '이준범', '김향미', '김영현', '새로이', '최세은', '홈마트',
       '이주희', '주아영', '가츠시', '황지혜', '차지이', '이장후', '유정화', '그리네', '배고팡',
       '윤지의', '이희진', '김한비', '박도형', '배창현', '김윤환', '팔공티', '이혜린', '정아현',
       '박병오', '띵카롱', '차송하', '박효정', '김은숙', '사계절', '아비꼬', '임소현', '백남문',
       '남한나', '송준희', '김유진', '한초원', '정명주', '한예진', '윤정은', '전대숙', '김승현',
       '세종원', '한은비', '손지연', '조정인', '김지오', '하오츠', '노유정', '카카오', '김진희',
       '심비어', '김희선', '장현진', '신민철', '변준영', '양주영', '양분식', '도네누', '카페롱',
       '천미향', '신동희', '김두열', '하얗다', '더호떡', '정홍비', '권규경', '이승연', '꼽사리',
       '박민서', '홍짱H', '연약국', '송승훈', '강양기', '시너리', '개미집', '차승현', '류영구',
       '이시현', '위메프', '김민지', '박태정', '민수현'], dtype=object)

In [168]:
exception_list=['새로이','홈마트','가츠시','그리네','배고팡','팔공티','띵카롱','아비꼬','도네누','카페롱','하얗다','더호떡','꼽사리','연약국','시너리','개미집','위메프']
drop_df = pd.DataFrame(train_df['기재내용'][drop1_index])
drop_df

,기재내용
7,김은희
11,진현영
12,이재성
14,이준범
18,김향미
...,...
1130,홈마트
1138,최세은
1139,최세은
1140,진현영


In [169]:
drop_final_index = drop_df['기재내용'][~drop_df['기재내용'].isin(exception_list)].index
train_df = train_df.drop(index=drop_final_index).reset_index().drop('index', axis=1)
train_df

,거래일시,기재내용,출금금액,Category
0,2019.04.25 14:13,파리바게트 화양리점,6150,NaN
1,2019.04.25 20:33,GS25군자행운점,10000,NaN
2,2019.04.26 02:22,비지에프리테일세종대,4000,NaN
3,2019.04.26 04:23,GS25화양중앙점,4320,NaN
4,2019.04.29 10:38,씨브이에스넷(주),5500,NaN
...,...,...,...,...
954,2020.04.22 08:47,(주)이랜드리테일,7800,NaN
955,2020.04.22 13:16,탕화쿵푸마라탕,11500,NaN
956,2020.04.24 19:40,네이버페이,9000,NaN
957,2020.04.25 02:34,Apple,3300,NaN


### 네이버페이, Apple 제거 

In [160]:
train_df['기재내용'][train_df['기재내용'].isin(['네이버페이', 'Apple'])].index

Int64Index([  40,   80,  103,  120,  133,  264,  300,  349,  370,  418,  641,
             709,  724,  755,  816,  855,  856,  902,  945,  953,  955,  963,
             992, 1024, 1030, 1031, 1033, 1041, 1066, 1067, 1072, 1079, 1086,
            1087, 1088, 1093, 1096, 1117, 1124, 1131, 1141, 1142, 1143],
           dtype='int64')

In [175]:
drop_final_index2 = train_df['기재내용'][train_df['기재내용'].isin(['네이버페이', 'Apple'])].index
train_df = train_df.drop(index=drop_final_index2).reset_index().drop('index', axis=1)
train_df

,거래일시,기재내용,출금금액,Category
0,2019.04.25 14:13,파리바게트 화양리점,6150,NaN
1,2019.04.25 20:33,GS25군자행운점,10000,NaN
2,2019.04.26 02:22,비지에프리테일세종대,4000,NaN
3,2019.04.26 04:23,GS25화양중앙점,4320,NaN
4,2019.04.29 10:38,씨브이에스넷(주),5500,NaN
...,...,...,...,...
911,2020.04.21 07:58,파리바게뜨군자역점,1200,NaN
912,2020.04.21 17:22,이마트24 광진코지웰,3800,NaN
913,2020.04.21 17:27,큰집닭강정,6000,NaN
914,2020.04.22 08:47,(주)이랜드리테일,7800,NaN


### 카테고리 크롤링

In [176]:
# 공백제거
train_df.기재내용 = train_df.기재내용.str.replace(' ', '')

# https://book.coalastudy.com/data_crawling/week6/stage2
# 네이버 지도 구버전
import time

driver = webdriver.Chrome("./chromedriver (2)")
categories = []

for i in range(10):
    driver.get("https://v4.map.naver.com")
    kword = train_df['기재내용'][i]
    driver.find_elements_by_css_selector("button.btn_close")[1].click()
    search_box  = driver.find_element_by_css_selector("input#search-input").send_keys(kword)
    search_button = driver.find_element_by_css_selector("button.spm").click()
    stores = driver.find_elements_by_css_selector("div.lsnx")

    for store in stores:
        category = store.find_element_by_css_selector("dd.cate").text
        
    categories.append(category)
    time.sleep(3)

In [177]:
categories

['카페,디저트 > 베이커리',
 '생활,편의 > 편의점',
 '생활,편의 > 편의점',
 '생활,편의 > 편의점',
 '화물운송 > 택배',
 '생활,편의 > 편의점',
 '생활,편의 > 편의점',
 '생활,편의 > 편의점',
 '생활,편의 > 편의점',
 '음식점 > 카페,디저트']

In [ ]:
from openpyxl import Workbook 
# 매번 크롤링해서 가져올 수 없으니까 한번 가져와서 엑셀에 저장하자

write_workbook = Workbook()
write_cell = write_workbook.active

for i in range(1, len(categories) +1):
    write_cell.cell(i, 1, categories[i-1]) # (1,1) (2,1) (3,1) ~~~ 에 추가
    
write_workbook.save('category_crawled.xlsx')

In [ ]:
# 저장한 엑셀 불러올 때
read_cell = read_workbook.active

category_list = []

for i in range(1, len(categories) :
    category_list.append(read_cell.cell(i, 1).value)
               
for i in result_list:
    print(i)